### Image Classification

### Import the Libraries

In [32]:
import numpy as np
import cv2
import os

from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier

### Unzip the dataset

Dataset diunzip terlebih dahulu.

In [33]:
import zipfile
with zipfile.ZipFile("/kaggle/input/dogs-vs-cats/train.zip","r") as z:
    z.extractall(".")
with zipfile.ZipFile("/kaggle/input/dogs-vs-cats/test1.zip","r") as z:
    z.extractall(".")

### Read the Dataset

Dataset dibaca dan disampling. Diambil 5000 data untuk training dan 1000 data untuk test.



In [34]:
size_train = 5000
size_test = 1000

data_path = '/kaggle/working/train'
label_dict = {'cat' : 0, 'dog' : 1} # Labeling, 0 untuk cat 1 untuk dog.

def read_data(label) : 
    images, labels = [], []
    label_size_dict = {0 : 0, 1 : 0}
    for idx, file_name in enumerate(os.listdir(data_path)) :
        if (label == 0 and idx == size_train ) or (label == 1 and idx == size_train + size_test) : 
                break
        if label == 1 and idx < size_train : 
            continue
        file_path = f"{data_path}/{file_name}"
        img = cv2.imread(file_path)
        string = file_name.split('.')[0]
        images.append(img)
        labels.append(label_dict[string])
        label_size_dict[label_dict[string]] += 1
    print(f"Size of cat : {label_size_dict[0]}\nSize of Dog : {label_size_dict[1]}")
    return images,labels

train_images, y_train = read_data(0)
test_images, y_test = read_data(1)

Size of cat : 2462
Size of Dog : 2538
Size of cat : 500
Size of Dog : 500


### Extract the Descriptors using SIFT

Deksriptor dari gambar lalu diekstrak dengan menggunakan SIFT

In [35]:
from tqdm import tqdm
def extract_sift_features(images):
    all_descriptors = []
    descriptors = []
    sift = cv2.SIFT_create()
    for img in tqdm(images):
        kp, des = sift.detectAndCompute(img,None)
        all_descriptors.extend(des)
        descriptors.append(des)
    return all_descriptors,descriptors

all_descriptors,train_descriptors = extract_sift_features(train_images) 
_,test_descriptors = extract_sift_features(test_images) 

100%|██████████| 1000/1000 [00:50<00:00, 19.61it/s]


### Clustering : Train the Model using KMeans

Model lalu di-train untuk diterapkan ke semua descriptors.

In [36]:
kmeans = KMeans(n_clusters = 75, max_iter=10,verbose=1,n_init=2)
kmeans.fit(all_descriptors)
cluster_centers = kmeans.cluster_centers_ 
print(cluster_centers)

Initialization complete
Iteration 0, inertia 509913791147.0
Iteration 1, inertia 353674978118.88025
Iteration 2, inertia 347753150114.9658
Iteration 3, inertia 345351531464.62115
Iteration 4, inertia 344003504085.0757
Iteration 5, inertia 343156250297.3355
Iteration 6, inertia 342590776778.3918
Iteration 7, inertia 342182731739.7987
Iteration 8, inertia 341861807533.7506
Iteration 9, inertia 341591208730.1713
Initialization complete
Iteration 0, inertia 510991355073.0
Iteration 1, inertia 354657900556.22394
Iteration 2, inertia 348326985730.34863
Iteration 3, inertia 345789674738.8419
Iteration 4, inertia 344358002053.739
Iteration 5, inertia 343407299148.8772
Iteration 6, inertia 342729993075.20996
Iteration 7, inertia 342232718010.2013
Iteration 8, inertia 341855814076.27124
Iteration 9, inertia 341560618428.91907
[[23.49504676 21.0100913  21.90631353 ... 12.275755    9.61976491
  18.56006728]
 [37.3568093  29.81419425 15.29516285 ... 60.4688684  19.86933537
   9.39661958]
 [ 6.19568

### Histogram Construction 

Features lalu di-assign ke cluster center, lalu histogram dari cluster assignments di bangun.

In [37]:
def features_to_vbow(descriptors) :
    vbow = []
    for descriptor in tqdm(descriptors) :
        hist = np.zeros(len(cluster_centers))
        indexes = kmeans.predict(descriptor.astype('float'))
        for idx in indexes : 
            hist[idx] += 1
        vbow.append(hist)
    return vbow

x_train = features_to_vbow(train_descriptors)
x_test = features_to_vbow(test_descriptors)

100%|██████████| 1000/1000 [00:02<00:00, 428.25it/s]


### Train the KNN Classifier

KNN Classifier lalu di-train dengan n = 3

In [38]:
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(x_train,y_train)

KNeighborsClassifier(n_neighbors=3)

### Predict

Dataset test lalu digunakan untuk melakukan prediksi

In [39]:
y_pred_test = knn.predict(x_test)
y_pred_test

array([1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0,
       1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1,
       1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0,
       0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1,
       0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1,
       0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0,
       1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0,
       1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1,
       0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1,
       0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0,
       0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1,

### Result

Hasil dari ```y_pred_test``` merupakan sebuah list yang berisi hasil prediksi dari dataset test. 

```0``` menandakan hasil prediksinya adalah cat <br>
```1``` menandakan hasil prediksinya adalah dog